# Geometrical Methods in Machine Learning
## Seminar 2: ICA

In [ ]:
!pip install nilearn

In [ ]:
from __future__ import print_function

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import math as m
from scipy import signal
from scipy.linalg import fractional_matrix_power as matrix_power

from sklearn.decomposition import PCA, FastICA
from sklearn.preprocessing import StandardScaler

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

from nilearn import datasets
from nilearn.input_data import NiftiMasker
from nilearn import image
from nilearn.plotting import plot_stat_map, show

## 1. ICA model

Consider ICA model:

$$\mathbf{X} = \mathbf{A}\mathbf{S}$$

where $\mathbf{S} \in \mathbb{R}^{n \times d}$ - $n$ source signals of dimension $d$, $\mathbf{X} \in \mathbb{R}^{m \times d}$ - $m$ observations of dimension $d$, $\mathbf{A}$ - $m \times n$ mixing matrix, where $m \geq n, \mathrm{Rank}(\mathbf{A}) = n$.

In [ ]:
np.random.seed(66)
S = np.random.logistic(0, 1, (2, 2000)) ** 3
S = S / S.std()
S[0, :] /= 4.

In [ ]:
fig = plt.figure(figsize=(8,4))

plt.subplot(121)
plt.xlim(-1.5, 1.5)
plt.xlabel("Value")
plt.ylabel("Probability")
plt.grid(linestyle="dotted")
plt.title("$X_1$")
plt.hist(S[0,:], bins=50, density=True)

plt.subplot(122)
plt.xlim(-1.5, 1.5)
plt.grid(linestyle="dotted")
plt.title("$X_2$")
plt.xlabel("Value")
plt.hist(S[1,:], bins=50, density=True)

plt.show()

#### Exercise

Set mixing matrix $A = \begin{pmatrix} 1 & 1 \\ 0 & 2 \end{pmatrix}$ and define X as linear mixture 

In [ ]:
# set mixing matrix A
A = # your code here

# set X as linear mixture of S
X = # your code here

In [ ]:
pca = PCA(n_components=2).fit(X.T)
ica = FastICA(n_components=2).fit(X.T)
print("PCA components:\n", pca.components_)
print("\nICA components:\n", ica.mixing_)

In [ ]:
ica_mixing_0 = ica.mixing_[:,0] / np.linalg.norm(ica.mixing_[:,0])
ica_mixing_1 = ica.mixing_[:,1] / np.linalg.norm(ica.mixing_[:,1])

In [ ]:
fig = plt.figure(figsize=(12,6))

plt.subplot(121)
plt.xlim(-6, 6)
plt.ylim(-6, 6)
plt.xlabel("X")
plt.ylabel("Y")
plt.grid(linestyle="dotted")
plt.title("True data")
plt.scatter(S.T[:,0], S.T[:,1], alpha=0.1)

plt.subplot(122)
plt.xlim(-6, 6)
plt.ylim(-6, 6)
plt.grid(linestyle="dotted")
plt.title("Mixed data")
plt.xlabel("X")
plt.scatter(X.T[:,0], X.T[:,1], alpha=0.1)

plt.quiver(0, 0, ica_mixing_0[0], ica_mixing_0[1], zorder=11, width=0.01, scale=5, color='r', alpha=0.8, label='ICA')
plt.quiver(0, 0, ica_mixing_1[0], ica_mixing_1[1], zorder=11, width=0.01, scale=5, color='r', alpha=0.8)

plt.quiver(0, 0, pca.components_[0,0], pca.components_[1,0], zorder=11, width=0.01, scale=5, color='orange', alpha=0.8, label='PCA')
plt.quiver(0, 0, pca.components_[0,1], pca.components_[1,1], zorder=11, width=0.01, scale=5, color='orange', alpha=0.8)

plt.legend()
plt.show()

## 2. Data whitening

Consider a generated dataset which is sampled from multivariate normal distribution with covariance matrix $C = \begin{pmatrix} 5 & 3 \\ 3 & 2 \end{pmatrix}$

In [ ]:
# sample size
sample_size = 1000

# sample from multivariate Gaussian
mu = np.random.normal(0, 0.5, 2)
C = np.array([[5, 3], [3, 2]])
data = np.random.multivariate_normal(mu, C, size=sample_size)

In [ ]:
idx = (data[:,0] > 3) & (data[:,1] > 3)

In [ ]:
fig = plt.figure(figsize=(6,6))
plt.xlim(-6, 6)
plt.ylim(-6, 6)
plt.grid(linestyle="dotted")

plt.title("Original data, mu={:.2f}, {:.2f}, var=1, 1".format(mu[0], mu[1]))
plt.xlabel("X")
plt.ylabel("Y")
plt.scatter(data[:,0], data[:,1], alpha=0.1)
plt.scatter(data[idx,0], data[idx,1], alpha=0.5)
plt.scatter(mu[0], mu[1], alpha=1, c='r')
plt.show()

Whitening procedure makes the samples made uncorrelated and their variances one.

Whitening has two simple steps:

1. (decorrelation) Project the dataset onto the eigenvectors (PCA). This rotates the dataset so that there is no correlation between the components.

2. (standartization) Normalize the the dataset to have a variance of 1 for all components. This is done by simply dividing each component by the square root of its eigenvalue.

Let $\mathbf{S} = \mathbf{V} \mathbf{\Lambda} \mathbf{V}^T$ is eigenvalue decomposition of sample covariance matrix $\mathbf{S} \triangleq \mathbf{X}^T\mathbf{X}$.

Then whitening matrix $\mathbf{W}$ is defined as:

$$\mathbf{W} = \mathbf{V} \mathbf{\Lambda}^{-1/2}$$

or alternatively (ZCA variant): 

$$\mathbf{W}_{ZCA} = \mathbf{V} \mathbf{\Lambda}^{-1/2} \mathbf{V}^T$$

Whitening transform is defined as:

$$\mathbf{X}' = \mathbf{W} \mathbf{X}$$

#### Exercise

Implement whitening matrix and whitening transform. Use either `scipy.linalg.fractional_matrix_power` or properties of diagonal matrix to raise it into power of $-1/2$.

In [ ]:
# get sample mean
x_mean = # your code here

# center data
data_centered = # your code here

# compute covariance matrix
covariance = # your code here
print("Covariance matrix:\n", covariance)

# obtain eigenvectors of covariance matrix
eigenvalues, eigenvectors = # your code here
print("\nEigenvalues:\n", eigenvalues)
print("\nEigenvectors:\n", eigenvectors)

# compute whitening matrix
W_zca = # your code here
print("\nZCA whitening matrix:\n", W_zca)

# compute PCA whitening matrix
W = # your code here
print("\nPCA whitening matrix:\n", W)

# scale data
data_scaled = StandardScaler().fit_transform(data)

# compute whitened data
data_whitened = np.dot(data_centered, W)
data_whitened_zca = np.dot(data_centered, W_zca)

#### Exercise

Check whether whitened data indeed have unit covariance.

In [ ]:
# your code here

In [ ]:
fig = plt.figure(figsize=(16,4))

plt.suptitle("Scaling and whitening", fontsize=13)

plt.subplot(141)
plt.xlim(-8, 8)
plt.ylim(-8, 8)
plt.xlabel("X")
plt.ylabel("Y")
plt.grid(linestyle="dotted")
plt.title("Original data")
plt.scatter(data[:,0], data[:,1], alpha=0.2)
plt.scatter(data[idx,0], data[idx,1], alpha=0.5)
plt.scatter(mu[0], mu[1], alpha=1, c='r')

plt.subplot(142)
plt.xlim(-5, 5)
plt.ylim(-5, 5)
plt.grid(linestyle="dotted")
plt.title("Scaled data")
plt.xlabel("X")
plt.scatter(data_scaled[:,0], data_scaled[:,1], alpha=0.2)
plt.scatter(data_scaled[idx,0], data_scaled[idx,1], alpha=0.5)
plt.scatter(np.mean(data_scaled, axis=0)[0], np.mean(data_scaled, axis=0)[1], alpha=1, c='r')

plt.subplot(144)
plt.xlim(-5, 5)
plt.ylim(-5, 5)
plt.grid(linestyle="dotted")
plt.title("PCA whitening")
plt.xlabel("X")
plt.scatter(data_whitened[:,0], data_whitened[:,1], alpha=0.2)
plt.scatter(data_whitened[idx,0], data_whitened[idx,1], alpha=0.5)
plt.scatter(np.mean(data_whitened, axis=0)[0], np.mean(data_whitened, axis=0)[1], alpha=1, c='r')

plt.subplot(143)
plt.xlim(-5, 5)
plt.ylim(-5, 5)
plt.grid(linestyle="dotted")
plt.title("ZCA whitening")
plt.xlabel("X")
plt.scatter(data_whitened_zca[:,0], data_whitened_zca[:,1], alpha=0.2)
plt.scatter(data_whitened_zca[idx,0], data_whitened_zca[idx,1], alpha=0.5)
plt.scatter(np.mean(data_whitened_zca, axis=0)[0], np.mean(data_whitened_zca, axis=0)[1], alpha=1, c='r')

plt.subplots_adjust(top = 0.82)

plt.show()

## 3. ICA solution

### Kurtosis approach - make the output signal components most non-Gaussian

ICA algorithm is conceptually relatively simple and works as follows:

1. remove the mean and whiten (decorrelate and normalize covariance matrix to unit variance) the data
2. rotate the data such that output signal components are most non-Gaussian.

Consider a linear mixture of two sources: $X_1 \sim Laplace(0,1)$ and $X_2 \sim U(0,1)$:

In [ ]:
# sample size and dimensionality
n, d = 1000, 2

X_1 = np.random.uniform(-np.sqrt(3),np.sqrt(3), (n))
X_2 = np.random.laplace(0, 0.5, (n))
X = np.vstack([X_1, X_2])
X.shape

In [ ]:
fig = plt.figure(figsize=(8,4))

plt.subplot(121)
#plt.xlim(-5, 5)
#plt.ylim(-5, 5)
plt.xlabel("Value")
plt.ylabel("Probability")
plt.grid(linestyle="dotted")
plt.title("$X_2 \sim U(0,1)$")
plt.hist(X_1, bins=50, density=True)

plt.subplot(122)
#plt.xlim(-5, 5)
#plt.ylim(-5, 5)
plt.grid(linestyle="dotted")
plt.title("$X_1 \sim Laplace(0,1)$")
plt.xlabel("Value")
plt.hist(X_2, bins=50, density=True)

plt.show()

#### Exercise

By defining a random mean and random mixing matrix $\mathbf{M}$ obtain mixed data.

In [ ]:
# set random mean
mean = # your code here

# create random mixing matrix
M = # your code here

# mix initial sources and add mean.
Y = # your code here

print("Initial sources:\n", X)
print("\nInitial mean:\n", mean)
print("\nMixing matrix:\n", M)
print("\nMixed data:\n", Y)

In [ ]:
fig = plt.figure(figsize=(6,6))

plt.xlabel("X")
plt.ylabel("Y")
plt.grid(linestyle="dotted")
plt.title("Mixed data")
plt.scatter(Y[0], Y[1], alpha=0.2)
plt.scatter(np.mean(Y.T, axis=0)[0], np.mean(Y.T, axis=0)[1], alpha=1)

plt.show()

In [ ]:
def white(data):
    x_mean = np.mean(data, axis=1)
    data_centered = data - x_mean.reshape((2, 1))
    covariance = data_centered.dot(data_centered.T) / n
    eigenvalues, eigenvectors = np.linalg.eig(covariance)
    W = eigenvectors.dot(matrix_power(np.diag(eigenvalues), -0.5)).dot(eigenvectors.T)
    return np.dot(W, data_centered), data_centered, W

**Step 1:** whiten the data

In [ ]:
Y_white, Y_centered, W = white(Y)
Y_white.shape, Y_centered.shape

In [ ]:
Y_white.dot(Y_white.T)

In [ ]:
fig = plt.figure(figsize=(12,6))

plt.subplot(121)
plt.xlim(-5, 5)
plt.ylim(-5, 5)
plt.xlabel("X")
plt.ylabel("Y")
plt.grid(linestyle="dotted")
plt.title("Whiten data")
plt.scatter(Y_white[0], Y_white[1], alpha=0.2)
plt.scatter(np.mean(Y_white, axis=1)[0], np.mean(Y_white, axis=1)[1], alpha=1)

plt.show()

### Maximizing measure of non-Gaussianity

Loss function (_contrast_ for historical reasons) is defined as:

$$C(X) := \sum_i \mathrm{kurt}(X)^2$$

Where _kurthosis_ (corrected four-order moment or cumulant) of (multivariate) random variable $X$ is defined:

$$\mathrm{kurt}(X) = \mathbb{E}(X^4) - 3\mathbb{E}(X^2)^2$$


#### Exercise

Code the kurtosis function.

In [ ]:
def kurtosis(data_centered):
    
    # your code here
    
    return kurtosis

In [ ]:
print(kurtosis(Y_white))

#### Exercise

The contrast function is a scalar function that quantifies on the basis of fourth order cumulants how non-Gaussian the components of a signal are.

Code the loss function.

In [ ]:
def loss(data_centered):
    
    # your code here
    
    return contrast

In [ ]:
print(loss(Y_white))

#### Exercise

**Step 2:** Find optimal rotation angle with random Givens rotations.

In higher dimensions a rotation matrix can be quite complex. To keep things simple, we use Givens rotations, which are defined as a rotation within the 2D subspace spanned by two axes n and m. For instance a  rotation matrix in 2D is given by:

$$\mathbf{R} = \begin{pmatrix} cos \phi & sin \phi \\ -sin \phi & cos \phi \end{pmatrix}$$

A rotation within the plane spanned by the second and the fourth axis ($n$ = 2, $m$ = 4) of a four-dimensional space is given by

$$\mathbf{R} = \begin{pmatrix} 1 & 0 & 0 & 0 \\ 0  & cos \phi & 0 & sin \phi \\ 0 & 0 & 1 & 0 \\ 0 & -sin \phi & 0 & cos \phi \end{pmatrix}$$

It can be shown that any general rotation, i.e. any orthogonal matrix with positive determinant, can be written as a product of Givens-rotations. Thus, we can find the general rotation matrix of the ICA problem by applying a series of Givens rotations and each time improve the objective function. By applying enough of such Givens-rotations, the algorithm should eventually converge to the globally optimal solution.

This is a simple procedure to rotate the data such that the contrast function is maximized, i.e. such that the components are maximally non-Gaussian, which implies that the components are minimally statistically independent (as far as fourth order is concerned).

In [ ]:
# Start with rotation angle 0, i.e. with rotation matrix [[1,0],[0,1]] and contrast of the whitened data.

bestAngle = 0
bestRotationMatrix = np.array([[1, 0], [0, 1]])
bestRotatedContrast = loss(Y_white)

losses = []

angle = 0
for iteration in np.arange(0, 10, 0.01):

    # Try a new rotation angle near the old one.
    
    # Vary the angle a bit, where 'a bit' gets exponentially smaller
    angle = angle + np.random.normal(0, m.exp(-iteration))
    
    # Calculate new rotation matrix
    rotationMatrix = np.array([[m.cos(angle), -m.sin(angle)], [m.sin(angle), m.cos(angle)]])
    
    # Calculate rotated whitened data
    rotatedData = np.dot(rotationMatrix, Y_white)
    
    # Calculate contrast ov newly rotated whitened data
    rotatedContrast = loss(rotatedData)
    
    # If contrast has improved, keep the new rotation angle, matrix, and contrast, otherwise continue with the old ones.
    if rotatedContrast > bestRotatedContrast:
        
        bestAngle = angle
        bestRotationMatrix = rotationMatrix
        bestRotatedContrast = rotatedContrast
        
        print("Loss (max): {:.4f}".format(rotatedContrast))
        
    losses.append(rotatedContrast)
        
# Display final result.
print("\nBest angle: ", bestAngle)
print("\nBest rotation matrix:\n", bestRotationMatrix)
print("\nBest rotated contrast: ", bestRotatedContrast)

In [ ]:
fig = plt.figure(figsize=(8,4))
plt.grid(linestyle="dotted")
plt.title("Loss function")
plt.xlabel("Iteration")
plt.plot(losses)

#### Exercise

Find unmixing matrix $\mathbf{U} = \mathbf{R} \mathbf{W}$ and separated sources $\mathbf{S} = \mathbf{U} \mathbf{\bar{Y}}$, where $\mathbf{\bar{Y}}$ is centered data.

In [ ]:
# find unmixing matrix U
U = # your code here

# find separated sources S
S = # your code here

In [ ]:
fig = plt.figure(figsize=(6,6))

plt.xlabel("$S_1$")
plt.ylabel("$S_2$")
plt.grid(linestyle="dotted")
plt.title("Separated data")
plt.scatter(S[0], S[1], alpha=0.2)
plt.scatter(np.mean(S, axis=1)[0], np.mean(S, axis=1)[1], alpha=1)

plt.show()

#### Exercise

Plot the distribution of separated sources

In [ ]:
fig = plt.figure(figsize=(8,4))

plt.subplot(121)
#plt.xlim(-5, 5)
#plt.ylim(-5, 5)
plt.xlabel("Value")
plt.ylabel("Probability")
plt.grid(linestyle="dotted")
plt.title("Separated 1")
plt.hist(S[0], bins=50, density=True)

plt.subplot(122)
#plt.xlim(-5, 5)
#plt.ylim(-5, 5)
plt.grid(linestyle="dotted")
plt.title("Separated 2")
plt.xlabel("Value")
plt.hist(S[1], bins=50, density=True)

plt.show()

## 4. Real data

### 4.1. Blind source separation

In [ ]:
# set image size
shape = (512, 512)
rows, cols = shape

# load images
img1 = np.load('./data/camera.npy').flatten()
img2 = np.load('./data/astronaut.npy').flatten()
img3 = np.load('./data/moon.npy').flatten()
img4 = np.load('./data/noise.npy').flatten()

# combine images
S = np.c_[img1, img2, img3, img4].T

In [ ]:
# set random mixing matrix A
A = np.random.uniform(0.01, 0.9, (4, 4))
# mix data
X = np.dot(A, S)
X.shape

In [ ]:
# show images
f, ax = plt.subplots(1, X.shape[0])
f.set_size_inches((16,4))

for i in range(X.shape[0]):
    ax[i].imshow(X[i,:].reshape(rows, cols), cmap=plt.gray())

#### Exercise

Run ICA with different number of `n_components`. Compare results, conclude.

In [ ]:
# run ICA
ica = # your code here
img_ica = # your code here
img_ica.shape

In [ ]:
# show images
f, ax = plt.subplots(1, img_ica.shape[1])
f.set_size_inches((16,4))

for i in range(img_ica.shape[1]):
    ax[i].imshow(img_ica[:,i].reshape(shape), cmap=plt.gray())

### 4.2. Time series

In [ ]:
np.random.seed(42)
n_samples = 2000
time = np.linspace(0, 8, n_samples)

s1 = np.sin(2 * time)  # Signal 1 : sinusoidal signal
s2 = np.sign(np.sin(3.5 * time))  # Signal 2 : square signal
s3 = signal.sawtooth(1.5 * np.pi * time) # sawtooteh

S = np.c_[s1, s2, s3]
#S += np.array([0, 0, np.random.normal()])  # Add noise

S /= S.std(axis=0)  # Standardize data

# Mix data
A = np.array([[1, 1, 1], [0.5, 2, 1.0], [1.5, 1.0, 2.0]])  # Mixing matrix
#A = np.random.normal(0, 1, (3, 3))
X = np.dot(S, A.T)  # Generate observations

# Compute ICA
ica = FastICA(n_components=3)
S_ = ica.fit_transform(X)  # Reconstruct signals
A_ = ica.mixing_  # Get estimated mixing matrix

# We can `prove` that the ICA model applies by reverting the unmixing.
#assert np.allclose(X, np.dot(S_, A_.T) + ica.mean_)

# For comparison, compute PCA
pca = PCA(n_components=3)
H = pca.fit_transform(X)  # Reconstruct signals based on orthogonal components

In [ ]:
fig = plt.figure(figsize=(12,6))

models = [X, S, S_, H]
names = ['Observations (mixed signal)',
         'True Sources',
         'ICA recovered signals',
         'PCA recovered signals']
colors = ['red', 'steelblue', 'orange']

for ii, (model, name) in enumerate(zip(models, names), 1):
    plt.subplot(4, 1, ii)
    plt.title(name)
    for sig, color in zip(model.T, colors):
        plt.plot(sig, color=color)

plt.subplots_adjust(0.09, 0.04, 0.94, 0.94, 0.26, 0.46)
plt.show()

## 5. fMRI data

The fMRI signal is represented by a $t \times v$ data matrix $\mathbf{X}$, where $t$ is the number of time points and $v$ is the number of voxels in the volumes. This means that each row of $\mathbf{S}$ contains an independent spatial pattern and the corresponding column of $\mathbf{A} \in \mathbb{R}^{t \times t}$ holds its activation time-course.

In [ ]:
from IPython.display import Image
Image("img/fmri.png")

In [ ]:
# fetch patient data
dataset = datasets.fetch_adhd(n_subjects=1)
func_filename = dataset.func[0]

# extract time series
masker = NiftiMasker(smoothing_fwhm=8, memory='nilearn_cache', detrend=True, standardize=True)
fmri_data = masker.fit_transform(func_filename)

fmri_data.shape

#### Exercise

Plot every 1000-th voxel activation time course.

In [ ]:
fig = plt.figure(figsize=(16,4))
plt.grid(linestyle="dotted")
plt.xlabel("Time step")

for i in range(0, fmri_data.shape[1], 1000):
    plt.plot(fmri_data[:,i])

#### Exercise

Fit ICA to fMRI data to extract desired number of independent components.

In [ ]:
n_components = 20

# apply ICA on spatial domain
ica = FastICA(n_components=n_components, random_state=42)
components = ica.fit_transform(fmri_data.T).T

In [ ]:
# normalize estimated components, for thresholding to make sense
components -= components.mean(axis=0)
components /= components.std(axis=0)

# threshold
components[np.abs(components) < 1.0] = 0

# now invert the masking operation, projecting to original space
component_img = masker.inverse_transform(components)

# plot image
mean_img = image.mean_img(func_filename)

for i in range(n_components):
    plot_stat_map(image.index_img(component_img, i))

#### Exercise

Fit PCA to fMRI data to extract desired number of principal components. Compare the result to ICA, conclude.

In [ ]:
### your code here